# To do:

 - Figure out how to signal injuries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import duckdb
import warnings
import math         # haversine_km()
import os

import xgboost as xgb
from xgboost import XGBRegressor
from scipy.stats import randint, uniform

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import joblib
import warnings
from datetime import datetime, timedelta
from haversine import haversine

pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

categories = ['PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK']
con = duckdb.connect(database=":memory:")

cwd = os.path.abspath(os.getcwd()).replace("\\", "/")
if cwd.startswith("C:/Users/Rodolfo/"):
    RUN_LOCATION = "local"
else:
    RUN_LOCATION = "cloud"
time_offset = {"local": 3, "cloud": -5}
now = str((datetime.now() + timedelta(hours=time_offset[RUN_LOCATION]) + timedelta(hours=-3)).date())
print(f"Today's date:", now)

tgt_stat = "PRA"
print('Target Stat:', tgt_stat)

Today's date: 2026-01-04
Target Stat: PRA


In [2]:
%run ./common_utils.ipynb

# ML Functions

In [3]:
def feature_importance(model):
    importance = model.get_score(importance_type='gain')

    # Convert to table
    df_importance = (
        pd.DataFrame({
            'feature': list(importance.keys()),
            'importance': list(importance.values())
        })
        .sort_values(by='importance', ascending=False)
        .reset_index(drop=True)
    )

    df_importance['pct'] = df_importance.importance.cumsum() / df_importance.importance.sum()
    if df_importance.shape[0] >= 50:
        with pd.option_context('display.max_rows', None):
            display(df_importance)
    else:
        display(df_importance)

    xgb.plot_importance(model)
    plt.show()

In [4]:
def create_baseline_model(df, pred_col, DFS):
    
    train_df, val_df, test_df = DFS

    if pred_col == 'MP':
        print('Minutes Model')
        feature_cols = [
            'MP_lst_gm',
            'MP_last_5_avg',
            'MP_last_10_avg',
            'starter', 'bench', 'reserve'
        ]
    else:
        print(f'{pred_col} Stats Model')
        feature_cols = [
            'MP_lst_gm',
            'MP_last_5_avg',
            'MP_last_10_avg',
            f'{pred_col}_last_3_avg', f'{pred_col}_last_5_avg', f'{pred_col}_last_10_avg',
            f'Def_{pred_col}', f'Def_L5_{pred_col}'
        ]
    
    print('Train:', len(train_df), '/ Validation:', len(val_df), '/ Test:', len(test_df))
    
    X_train, y_train = train_df[feature_cols], train_df[pred_col]
    X_val,   y_val   = val_df[feature_cols],   val_df[pred_col]
    X_test,  y_test  = test_df[feature_cols],  test_df[pred_col]

    # Convert to DMatrix (XGBoost internal format)
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dval   = xgb.DMatrix(X_val, label=y_val)
    dtest  = xgb.DMatrix(X_test, label=y_test)

    params = {
        "objective": "reg:squarederror",
        "max_depth": 5,
        "learning_rate": 0.05,
        "subsample": 0.8,
        "colsample_bytree": 0.8,
        "seed": 42
    }

    # Train using native XGBoost API with early stopping
    evals = [(dtrain, "train"), (dval, "val")]
    bst = xgb.train(
        params,
        dtrain,
        num_boost_round=500,
        evals=evals,
        early_stopping_rounds=50,
        verbose_eval=False
    )

    # Predict on test set
    preds = bst.predict(dtest)

    rmse = np.sqrt(mean_squared_error(y_test, preds))
    mae = mean_absolute_error(y_test, preds)
    r2 = r2_score(y_test, preds)

    print("RMSE:", rmse)
    print("MAE:", mae)
    print("R²:", r2)
    
    return bst

In [5]:
def hyperparam_tuning(DFS, pred_col, n_iter=20, early_stopping_rounds=50):
    """
    Hyperparameter tuning using native XGBoost API and DMatrix,
    with early stopping support (compatible with XGBoost 3.1.2)
    """

    train_df, val_df, test_df = DFS
    feature_cols = [col for col in train_df.columns if col not in ['Date', pred_col]]
    X_train, y_train = train_df[feature_cols], train_df[pred_col]
    X_val,   y_val   = val_df[feature_cols],   val_df[pred_col]
    X_test,  y_test  = test_df[feature_cols],  test_df[pred_col]

    # Convert datasets to DMatrix
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dval   = xgb.DMatrix(X_val, label=y_val)
    dtest  = xgb.DMatrix(X_test, label=y_test)

    # Hyperparameter search space
    param_dist = {
        "n_estimators": randint(300, 1500),
        "learning_rate": uniform(0.01, 0.05),
        "max_depth": randint(3, 6),
        "min_child_weight": randint(1, 8),
        "subsample": uniform(0.7, 0.3),
        "colsample_bytree": uniform(0.7, 0.3),
        "gamma": uniform(0, 2),
        "reg_lambda": uniform(0, 5),
        "reg_alpha": uniform(0, 2)
    }

    # Sample n_iter random parameter combinations
    param_list = []
    for _ in range(n_iter):
        sample = {k: (v.rvs() if hasattr(v, "rvs") else v) for k, v in param_dist.items()}
        sample['n_estimators'] = int(sample['n_estimators'])
        sample['max_depth'] = int(sample['max_depth'])
        sample['min_child_weight'] = int(sample['min_child_weight'])
        param_list.append(sample)

    best_mae = float('inf')
    best_params = None
    best_bst = None

    # Manual hyperparameter search
    for i, params in enumerate(param_list):
        print(f"\nTrial {i+1}/{n_iter}: {params}")
        num_boost_round = params.pop('n_estimators')
        params.update({
            "objective": "reg:squarederror",
            "tree_method": "hist",
            "device": "cuda",
            "seed": 42
        })
        evals = [(dtrain, 'train'), (dval, 'val')]
        bst = xgb.train(
            params,
            dtrain,
            num_boost_round=num_boost_round,
            evals=evals,
            early_stopping_rounds=early_stopping_rounds,
            verbose_eval=False
        )
        # Predict on validation set to compute MAE
        val_preds = bst.predict(dval, iteration_range=(0, bst.best_iteration))
        mae = mean_absolute_error(y_val, val_preds)
        print(f"Validation MAE: {mae:.4f}")
        if mae < best_mae:
            best_mae = mae
            best_params = params.copy()
            best_bst = bst

    print("\nBest validation MAE:", best_mae)
    print("Best parameters:", best_params)

    # Predict on test set using best model
    preds = best_bst.predict(dtest, iteration_range=(0, best_bst.best_iteration))
    test_df[pred_col] = y_test
    test_df[f'{pred_col}_preds'] = preds
    test_df['Team'] = team_encoder.inverse_transform(test_df["Team"])
    test_df['Opp'] = team_encoder.inverse_transform(test_df["Opp"])
    test_df['Player'] = player_encoder.inverse_transform(test_df["Player"])
    test_df['Pos'] = position_encoder.inverse_transform(test_df["Pos"])
    analyze_df = test_df[['Date', 'Team', 'Player', 'Pos', 'Opp', pred_col, f'{pred_col}_preds']]
    print("\nTest Metrics:")
    print("RMSE:", np.sqrt(mean_squared_error(y_test, preds)))
    print("MAE:", mean_absolute_error(y_test, preds))
    print("R²:", r2_score(y_test, preds))

    return best_bst, preds, y_test, analyze_df

### Create Base df

In [63]:
df = pd.DataFrame()
df2 = pd.DataFrame()
df3 = pd.DataFrame()
df4 = pd.DataFrame()
for i in [2021, 2022, 2023, 2024, 2025]:
    df_actuals = pd.read_csv(f"../tables/{i}/parlay_stats.csv")
    df_actuals['Season'] = i
    df = pd.concat([df, df_actuals])

    df_schd = pd.read_csv(f"../tables/{i}/nba_schedule.csv")
    df_schd['Season'] = i
    df2 = pd.concat([df2, df_schd])
    
    df_gms = pd.read_csv(f"../tables/{i}/season_gamelogs.csv")
    df_gms['Season'] = i
    df3 = pd.concat([df3, df_gms])
    
    df_inj = pd.read_csv(f"../tables/{i}/injuries.csv")
    df_inj['Season'] = i
    df4 = pd.concat([df4, df_inj])

df['Date'] = pd.to_datetime(df.Date)
df2['Date'] = pd.to_datetime(df2.Date)
df3['Date'] = pd.to_datetime(df3.Date)
df3 = df3[~df3[['Date', 'Team', 'Player']].duplicated(keep='last')]
df4['Date'] = pd.to_datetime(df4.Date)

df3 = df3.rename(columns={"3PM": "TPM", "3PA": "TPA", "3P%": "TP%", "TRB": "REB"}).drop(['Pos', 'Opp'], axis=1)
df3['PR'] = df3.PTS + df3.REB 
df3['PA'] = df3.PTS + df3.AST
df3['RA'] = df3.REB + df3.AST
df3['PRA'] = df3.PTS + df3.REB + df3.AST
df3['STL_BLK'] = df3.STL + df3.BLK
df = df.merge(df3, on=['Season', 'Date', 'Team', 'Player'], how='left')

df_mtch = df2[['Season', 'Date', 'AwayABV', 'HomeABV', 'AwayPTS', 'HomePTS', 'AwayB2B', 'HomeB2B', 'is_OT', 'cup_gm', 'pstszn_gm']]
df_mtch['Team_type'] = 'Away'
df_mtch = df_mtch.rename(columns={"AwayABV": "Team", "HomeABV": "Opp", "AwayB2B": "B2B"})[['Season', 'Date', 'Team', 'AwayPTS', 'HomePTS', 'Opp', 'B2B', 'is_OT', 'cup_gm', 'pstszn_gm', 'Team_type']]
df_mtch2 = df_mtch.copy().rename(columns={"Team": "Opp", "Opp": "Team", "HomeB2B": "B2B"})[['Season', 'Date', 'Team', 'AwayPTS', 'HomePTS', 'Opp', 'B2B', 'is_OT', 'cup_gm', 'pstszn_gm']]
df_mtch2['Team_type'] = 'Home'
df_mtch = pd.concat([df_mtch, df_mtch2])
df_mtch = df_mtch[['Season', 'Date', 'Team', 'Team_type', 'AwayPTS', 'HomePTS', 'is_OT', 'cup_gm', 'pstszn_gm']]
df_mtch = df_mtch.sort_values(["Team", "Date"])
df_mtch['team_game_num'] = df_mtch.groupby(["Team", "Season"]).cumcount() + 1
df_mtch['Spread'] = np.where(df_mtch.Team_type == 'Home', df_mtch.HomePTS - df_mtch.AwayPTS, df_mtch.AwayPTS - df_mtch.HomePTS)
df_mtch['Total'] = df_mtch.AwayPTS + df_mtch.HomePTS
df_mtch['is_Win'] = np.where(df_mtch.Spread > 0, 1, 0)
df_mtch['Szn_Wins'] = df_mtch.groupby(['Season', 'Team'])['is_Win'].cumsum()
df = df.drop(['Season', 'Team_type'], axis=1).merge(df_mtch, on=['Date', 'Team'])

df = df.merge(df4[['Date', 'Team', 'Player', 'Status']], on=['Date', 'Team', 'Player'], how='left')
df['Status'] = np.where((df.Active == 1) & (df.Status.isnull()), 'Available', df.Status)
df['Status'] = np.where((df.Active == 0), 'Out', df.Status)
df['Status'] = np.where((df.Status == 'Out') & (df.Active != 0), 'Available', df.Status)

team_encoder = LabelEncoder()
player_encoder = LabelEncoder()
team_type_encoder = LabelEncoder()
position_encoder = LabelEncoder()
status_encoder = LabelEncoder()

# Encode string cols
team_encoder.fit(pd.concat([df["Team"], df["Opp"]], axis=0))
df["Team"] = team_encoder.transform(df["Team"])
df["Opp"] = team_encoder.transform(df["Opp"])
df["Player_name"] = df.Player
df["Player"] = player_encoder.fit_transform(df["Player"])
df["Pos"] = position_encoder.fit_transform(df["Pos"])
df['Team_type'] = team_type_encoder.fit_transform(df['Team_type'])
df["Status"] = status_encoder.fit_transform(df["Status"])
df_pred = df.copy()
df = df[(df.Active == 1) & (df.MP > 0)].sort_values(['Season', 'Date', 'Team', 'Player']).reset_index(drop=True)
print('base df created', datetime.now())

base df created 2026-01-04 15:17:37.975424


In [8]:
# df3_temp = df3.copy().drop('Season', axis=1)
# df4_temp = df4.copy().drop('Season', axis=1)
# # display(df3_temp[(df3_temp.Date == '2025-12-25') & (df3_temp.game_id == '20251225_CLE_NYK')])
# df_temp = df4_temp.merge(df3_temp, on=['Date', 'Team', 'Player'], how='outer')
# df_temp['game_id'] = np.where(df_temp.game_id.isnull(), )

# df_temp = df_temp[(df_temp.Date == '2025-12-25') & (df_temp.Team.isin(['CLE', 'NYK']))]
# display(df_temp)

# Minutes Projection Model

In [9]:
def haversine_km(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth radius in km
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat/2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def travel_km_from_row(row):
    prev = row['PrevLocation']
    cur  = row['Location']
    # missing prev => first game => no travel
    if pd.isna(prev) or pd.isna(cur):
        return 0.0
    # same arena => 0
    if prev == cur:
        return 0.0
    # lookup coords
    prev_coords = arenas.get(prev)
    cur_coords  = arenas.get(cur)
    if not prev_coords or not cur_coords:
        # fallback if code not found
        return 0.0
    return haversine_km(prev_coords[0], prev_coords[1], cur_coords[0], cur_coords[1])

In [69]:
def setup_df_mins(con, df):
    
    df = df[['Season', 'Date', 'Team', 'Team_type', 'Opp', 'Player', 'Pos', 'B2B', 'MP', 'TOV', 'PF', '+/-',
             'Spread', 'Total', 'team_game_num', 'Szn_Wins', 'cup_gm', 'pstszn_gm', 'is_OT']]
    
    for col in ['MP', 'TOV', 'PF', '+/-']:
        df[f'{col}_lst_gm'] = (
            df
            .groupby(['Player', 'Season'])[col]
            .shift(1)
        )
        for N in [3, 5, 10]:
            df[f'{col}_last_{N}_avg'] = (
                df.groupby(['Player', 'Season'])[col]
                  .rolling(window=N, min_periods=1)
                  .mean()
                  .shift(1)
                  .reset_index(level=[0, 1], drop=True)
            )
            df[f"{col}_last_{N}_std"] = (
                df.groupby(['Player', 'Season'])[col]
                  .shift(1)
                  .rolling(window=N, min_periods=1)
                  .std()
            )
        df[f"{col}_change_L1"] = df[f"{col}_lst_gm"] - df[f"{col}_last_5_avg"]
        df[f"{col}_change_L3"] = df[f"{col}_last_3_avg"] - df[f"{col}_last_10_avg"]
        df[f"{col}_pct_change"] = (
            (df[f"{col}_lst_gm"] - df[f"{col}_last_10_avg"]) /
            (df[f"{col}_last_10_avg"] + 1e-6)
        )
    df["MP_spike"] = (df["MP_lst_gm"] > df["MP_last_10_avg"] + 8).astype(int)
    df["MP_drop"]  = (df["MP_lst_gm"] < df["MP_last_10_avg"] - 8).astype(int)
    df["MP_trend"] = df["MP_last_3_avg"] - df["MP_last_10_avg"]

    games_last_7_days = df.groupby(['Player', 'Season']).rolling('7D', on='Date')['MP'].count().shift(1).to_frame(name='games_last_7_days').reset_index()
    games_last_7_days = games_last_7_days.drop_duplicates(
        subset=['Player', 'Season', 'Date']
    )
    df = df.merge(games_last_7_days, on=['Player', 'Season', 'Date'])
    df['games_last_7_days'] = df.games_last_7_days.fillna(0).astype(int)
    
    df['prev_team_mins_pct'] = (df.groupby(['Player', 'Season'])['MP'].shift(1)) / 240
           
    df['reserve_td'] = (df.MP < 8).astype(int)
    df['bench_td']   = ((df.MP >= 8) & (df.MP <= 25)).astype(int)
    df['starter_td'] = (df.MP > 25).astype(int)
    role_counts = df.groupby(['Season', 'Player'])[['reserve_td', 'bench_td', 'starter_td']].sum()
    role_counts['most_common_role'] = role_counts[['reserve_td', 'bench_td', 'starter_td']].idxmax(axis=1)
    role_counts['reserve'] = (role_counts['most_common_role'] == 'reserve_td').astype(int)
    role_counts['bench']   = (role_counts['most_common_role'] == 'bench_td').astype(int)
    role_counts['starter'] = (role_counts['most_common_role'] == 'starter_td').astype(int)
    df = df.merge(role_counts[['reserve', 'bench', 'starter']], on=['Season', 'Player'], how='left')
    
    df['role'] = 0
    df['role'] = np.where(df.starter == 1, 1, df.role)
    df['role'] = np.where(df.bench == 1, 2, df.role)
    df['role'] = np.where(df.reserve == 1, 3, df.role)
      
    df['missed_games'] = (
        df.groupby(['Player', 'Team', 'Season'])['team_game_num']      
          .diff()
          .sub(1)
          .fillna(0)
          .astype(int)
    )
    
    df['blowout'] = np.where(abs(df.Spread >= 15), 1, 0)
    df['Injury_Ejection'] = (df['MP'] < 5) | (df['MP_drop'] > 20) | ((df.MP_last_10_avg - df.MP) >= 20)
    
    # Location based features
    df["PrevOpp"] = df.groupby("Player")["Opp"].shift(1)
    df["DaysLstGm"] = (df.groupby("Player")["Date"].diff().dt.days).fillna(0).astype(int)
    df['Location'] = df.apply(lambda r: r['Team'] if r['Team_type'] == 'Home' else r['Opp'], axis=1)
    df['PrevLocation'] = df.groupby('Player')['Location'].shift(1)
    df['travel_km'] = df.apply(travel_km_from_row, axis=1).fillna(0)
    df['travel_hours'] = df['travel_km'] / 800.0      # approximate flight hours
    df['is_long_trip'] = (df['travel_km'] > 1500).astype(int)
    df['same_arena'] = (df['PrevLocation'] == df['Location']).astype(int)
    
    df = df.drop(['Season', 'reserve_td', 'reserve', 'bench_td', 'bench', 'starter_td', 'starter', 'Szn_Wins', 'TOV', 'PF', '+/-', 
                  'PrevOpp', 'PrevLocation', 'Location'], axis=1)    
    
    return df

In [ ]:
df_mins = df.copy()
df_mins = setup_df_mins(con, df_mins)
display(df_mins)

n = len(df_mins)
train_end = int(0.8 * n)
val_end   = int(0.9 * n)
mins_train_df = df_mins.iloc[:train_end]
mins_val_df   = df_mins.iloc[train_end:val_end]
mins_test_df  = df_mins.iloc[val_end:]
mins_DFS = (mins_train_df, mins_val_df, mins_test_df)

# mins_model = create_baseline_model(df_mins, "MP", mins_DFS)
mins_model, mins_preds, y_test_mins, analyze_df_mins = hyperparam_tuning(mins_DFS, "MP", n_iter=25)
# feature_importance(mins_model)

mins_model.save_model("../ML_models/mins_model.json")
print('Saved minutes model!')

,Date,Team,Team_type,Opp,Player,Pos,B2B,MP,Spread,Total,team_game_num,cup_gm,pstszn_gm,is_OT,MP_lst_gm,MP_last_3_avg,MP_last_3_std,MP_last_5_avg,MP_last_5_std,MP_last_10_avg,MP_last_10_std,MP_change_L1,MP_change_L3,MP_pct_change,TOV_lst_gm,TOV_last_3_avg,TOV_last_3_std,TOV_last_5_avg,TOV_last_5_std,TOV_last_10_avg,TOV_last_10_std,TOV_change_L1,TOV_change_L3,TOV_pct_change,PF_lst_gm,PF_last_3_avg,PF_last_3_std,PF_last_5_avg,PF_last_5_std,PF_last_10_avg,PF_last_10_std,PF_change_L1,PF_change_L3,PF_pct_change,+/-_lst_gm,+/-_last_3_avg,+/-_last_3_std,+/-_last_5_avg,+/-_last_5_std,+/-_last_10_avg,+/-_last_10_std,+/-_change_L1,+/-_change_L3,+/-_pct_change,MP_spike,MP_drop,MP_trend,games_last_7_days,prev_team_mins_pct,role,missed_games,blowout,Injury_Ejection,DaysLstGm,travel_km,travel_hours,is_long_trip,same_arena
0,2021-10-19,2,0,16,71,0,0,22.98,-23.0,231.0,1,0,0,0,NaN,5.410000,NaN,3.964,NaN,6.374000,NaN,NaN,-0.964000,NaN,NaN,0.333333,NaN,0.2,NaN,0.300000,NaN,NaN,0.033333,NaN,NaN,0.333333,NaN,0.2,NaN,1.000000,NaN,NaN,-0.666667,NaN,NaN,-2.333333,NaN,-1.0,NaN,-3.0,NaN,NaN,0.666667,NaN,0,0,-0.964000,2,NaN,2,0,0,False,0,0.0,0.0,0,0
1,2021-10-19,2,0,16,96,3,0,3.75,-23.0,231.0,1,0,0,0,NaN,18.450000,NaN,24.864,NaN,21.474000,NaN,NaN,-3.024000,NaN,NaN,0.666667,NaN,1.0,NaN,1.100000,NaN,NaN,-0.433333,NaN,NaN,0.666667,NaN,1.4,NaN,1.700000,NaN,NaN,-1.033333,NaN,NaN,-0.333333,NaN,3.2,NaN,1.5,NaN,NaN,-1.833333,NaN,0,0,-3.024000,4,NaN,1,0,0,True,0,0.0,0.0,0,0
2,2021-10-19,2,0,16,112,4,0,3.75,-23.0,231.0,1,0,0,0,NaN,20.623333,NaN,23.934,NaN,27.867000,NaN,NaN,-7.243667,NaN,NaN,2.000000,NaN,2.0,NaN,2.100000,NaN,NaN,-0.100000,NaN,NaN,1.000000,NaN,1.2,NaN,1.900000,NaN,NaN,-0.900000,NaN,NaN,4.333333,NaN,5.6,NaN,6.9,NaN,NaN,-2.566667,NaN,0,0,-7.243667,3,NaN,2,0,0,True,0,0.0,0.0,0,0
3,2021-10-19,2,0,16,211,3,0,3.75,-23.0,231.0,1,0,0,0,NaN,11.650000,NaN,11.650,NaN,11.650000,NaN,NaN,0.000000,NaN,NaN,0.500000,NaN,0.5,NaN,0.500000,NaN,NaN,0.000000,NaN,NaN,0.500000,NaN,0.5,NaN,0.500000,NaN,NaN,0.000000,NaN,NaN,-13.000000,NaN,-13.0,NaN,-13.0,NaN,NaN,0.000000,NaN,0,0,0.000000,2,NaN,2,0,0,True,0,0.0,0.0,0,0
4,2021-10-19,2,0,16,406,2,0,30.63,-23.0,231.0,1,0,0,0,NaN,4.620000,NaN,8.382,NaN,7.942857,NaN,NaN,-3.322857,NaN,NaN,0.333333,NaN,0.4,NaN,0.285714,NaN,NaN,0.047619,NaN,NaN,0.333333,NaN,1.0,NaN,0.714286,NaN,NaN,-0.380952,NaN,NaN,-2.666667,NaN,-1.8,NaN,0.0,NaN,NaN,-2.666667,NaN,0,0,-3.322857,3,NaN,1,0,0,False,0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112505,2026-01-03,28,0,9,581,0,0,18.67,-9.0,237.0,34,0,0,0,28.42,21.460000,11.197055,21.984,9.038716,22.710000,9.867945,6.436,-1.250000,0.251431,4.0,2.333333,1.527525,1.8,2.073644,1.400000,2.024846,2.2,0.933333,1.857142,5.0,4.000000,2.516611,3.0,1.816590,2.500000,1.563472,2.0,1.500000,1.000000,-18.0,-11.000000,13.453624,-9.0,10.134101,-2.7,10.174041,-9.0,-8.300000,5.666669,0,0,-1.250000,4,0.118417,2,0,0,False,2,0.0,0.0,0,0
112506,2026-01-03,28,0,9,595,1,0,40.75,-9.0,237.0,34,0,0,0,38.32,36.966667,5.777344,37.376,9.617337,35.120000,9.055767,0.944,1.846667,0.091116,0.0,1.000000,2.081666,1.4,2.073644,1.400000,2.024846,-1.4,-0.400000,-0.999999,1.0,2.666667,2.645751,3.0,1.923538,2.200000,1.563472,-2.0,0.466667,-0.545454,13.0,9.000000,16.643317,9.0,14.117365,1.5,11.862171,4.0,7.500000,7.666662,0,0,1.846667,3,0.159667,1,1,0,False,4,0.0,0.0,0,0
112507,2026-01-03,28,0,9,843,3,0,27.88,-9.0,237.0,34,0,0,0,23.62,19.653333,7.495999,19.102,9.638105,21.901000,9.100731,4.518,-2.247667,0.078490,0.0,0.333333,2.309401,0.4,1.816590,0.800000,2.131770,-0.4,-0.466667,-0.999999,1.0,0.666667,2.309401,0.6,1.923538,0.900000,1.563472,0.4,-0.233333,0.111111,-13.0,0.333333,16.643317,-5.0,13.773162,-4.7,11.117554,-8.0,5.033333,1.765958,0,0,-2.247667,3,0.098417,2,0,0,False,2,0.0,0.0,0,0
112508,2026-01-03,2


Trial 1/25: {'n_estimators': 423, 'learning_rate': np.float64(0.022195487595061767), 'max_depth': 5, 'min_child_weight': 6, 'subsample': np.float64(0.9435528543579819), 'colsample_bytree': np.float64(0.7538765115340633), 'gamma': np.float64(1.2947725622291872), 'reg_lambda': np.float64(1.9293158591876214), 'reg_alpha': np.float64(0.09788607196807186)}
Validation MAE: 4.2883

Trial 2/25: {'n_estimators': 724, 'learning_rate': np.float64(0.05465551262466637), 'max_depth': 5, 'min_child_weight': 7, 'subsample': np.float64(0.9183649925517988), 'colsample_bytree': np.float64(0.8729548706354021), 'gamma': np.float64(1.1635190251688836), 'reg_lambda': np.float64(4.126396727211271), 'reg_alpha': np.float64(1.3075809808083412)}
Validation MAE: 4.2782

Trial 3/25: {'n_estimators': 516, 'learning_rate': np.float64(0.04095484081220125), 'max_depth': 5, 'min_child_weight': 6, 'subsample': np.float64(0.9932446345099453), 'colsample_bytree': np.float64(0.9964987940224255), 'gamma': np.float64(0.5411

Validation MAE: 4.2723

Trial 23/25: {'n_estimators': 1497, 'learning_rate': np.float64(0.0540952319662028), 'max_depth': 4, 'min_child_weight': 1, 'subsample': np.float64(0.8306937772888143), 'colsample_bytree': np.float64(0.9475226008314693), 'gamma': np.float64(1.3137841040752025), 'reg_lambda': np.float64(0.45891853304246444), 'reg_alpha': np.float64(1.0403218996899664)}
Validation MAE: 4.2871

Trial 24/25: {'n_estimators': 567, 'learning_rate': np.float64(0.023007529472772514), 'max_depth': 4, 'min_child_weight': 1, 'subsample': np.float64(0.7417359947816886), 'colsample_bytree': np.float64(0.7070282567502508), 'gamma': np.float64(1.8638467431601664), 'reg_lambda': np.float64(1.904718607089782), 'reg_alpha': np.float64(1.454652857587747)}
Validation MAE: 4.2964

Trial 25/25: {'n_estimators': 1284, 'learning_rate': np.float64(0.010260962546830529), 'max_depth': 5, 'min_child_weight': 6, 'subsample': np.float64(0.8887398726624659), 'colsample_bytree': np.float64(0.7407259860865552),

In [60]:
analyze_df_mins = mins_test_df.drop(['MP', 'MP_preds'], axis=1)\
                .merge(analyze_df_mins[['Date', 'Team', 'Player', 'MP', 'MP_preds']], on=['Date', 'Team', 'Player'])
analyze_df_mins['Diff'] = analyze_df_mins['MP'] - analyze_df_mins[f'MP_preds']
analyze_df_mins['Diff2'] = abs(analyze_df_mins['Diff'])
analyze_df_mins.sort_values('Diff2', ascending=False).drop('Diff2', axis=1).head(15)

,Date,Team,Team_type,Opp,Player,Pos,B2B,Spread,Total,team_game_num,cup_gm,pstszn_gm,is_OT,MP_lst_gm,MP_last_3_avg,MP_last_3_std,MP_last_5_avg,MP_last_5_std,MP_last_10_avg,MP_last_10_std,MP_change_L1,MP_change_L3,MP_pct_change,TOV_lst_gm,TOV_last_3_avg,TOV_last_3_std,TOV_last_5_avg,TOV_last_5_std,TOV_last_10_avg,TOV_last_10_std,TOV_change_L1,TOV_change_L3,TOV_pct_change,PF_lst_gm,PF_last_3_avg,PF_last_3_std,PF_last_5_avg,PF_last_5_std,PF_last_10_avg,PF_last_10_std,PF_change_L1,PF_change_L3,PF_pct_change,+/-_lst_gm,+/-_last_3_avg,+/-_last_3_std,+/-_last_5_avg,+/-_last_5_std,+/-_last_10_avg,+/-_last_10_std,+/-_change_L1,+/-_change_L3,+/-_pct_change,MP_spike,MP_drop,MP_trend,games_last_7_days,prev_team_mins_pct,role,missed_games,blowout,Injury_Ejection,DaysLstGm,travel_km,travel_hours,is_long_trip,same_arena,MP,MP_preds,Diff
1202,2025-10-23,IND,1,OKC,Ben Sheppard,SG,0,-6.0,276.0,1,0,0,2,NaN,14.316667,NaN,14.522,13.074404,15.155000,11.278335,NaN,-0.838333,NaN,NaN,0.666667,NaN,0.6,1.414214,0.500000,1.496026,NaN,0.166667,NaN,NaN,1.000000,NaN,1.40,0.707107,1.700000,1.290994,NaN,-0.700000,NaN,NaN,-3.666667,NaN,-4.60,8.485281,-5.000000,10.954451,NaN,1.333333,NaN,0,0,-0.838333,3,NaN,2,0,0,False,123,0.0,0.0,0,1,39.05,13.424125,25.625875
8970,2025-12-19,MIA,0,BOS,Kasparas Jakucionis,PG,1,-13.0,245.0,28,0,0,0,7.63,4.265000,11.238756,4.265,11.798154,4.265000,9.182534,3.365,0.000000,0.788980,1.0,0.500000,1.527525,0.5,1.341641,0.500000,1.286684,0.5,0.000000,0.999998,0.0,0.000000,0.577350,0.00,2.167948,0.000000,1.646545,0.00,0.000000,0.000000,-4.0,-3.000000,4.932883,-3.00,5.700877,-3.000000,8.538150,-1.00,0.000000,0.333333,0,0,0.000000,1,0.031792,3,0,0,False,1,0.0,0.0,0,0,35.60,12.451878,23.148122
8486,2025-12-14,MIN,1,SAC,Bones Hyland,PG,0,14.0,220.0,26,0,0,0,4.65,7.303333,12.992753,7.860,11.237368,6.316000,9.531455,-3.210,0.987333,-0.263774,0.0,1.333333,1.527525,0.8,1.414214,0.400000,1.269296,-0.8,0.933333,-0.999998,1.0,1.000000,2.000000,0.80,1.483240,0.500000,1.779513,0.20,0.500000,0.999998,2.0,2.666667,6.557439,3.80,8.532292,1.800000,9.255029,-1.80,0.866667,0.111111,0,0,0.987333,2,0.019375,2,0,0,False,2,0.0,0.0,0,0,35.68,13.187209,22.492791
7426,2025-12-04,GSW,0,PHI,Draymond Green,PF,0,-1.0,197.0,23,0,0,0,28.85,31.490000,0.261630,31.968,5.132857,29.099000,5.828490,-3.118,2.391000,-0.008557,3.0,2.666667,2.121320,2.8,1.825742,3.400000,1.424001,0.2,-0.733333,-0.117647,5.0,4.000000,2.828427,4.00,1.914854,4.100000,1.301708,1.00,-0.100000,0.219512,-17.0,-1.666667,3.535534,0.20,5.057997,2.400000,8.645808,-17.20,-4.066667,-8.083330,0,0,2.391000,3,0.120208,1,0,0,False,2,0.0,0.0,0,0,9.38,31.788517,-22.408517
6891,2025-11-30,PHI,1,ATL,VJ Edgecombe,SG,0,-8.0,276.0,19,0,0,2,38.22,37.546667,2.333931,36.948,12.363159,35.974000,10.991925,1.272,1.572667,0.062434,1.0,1.666667,2.309401,1.4,1.788854,2.000000,1.429841,-0.4,-0.333333,-0.500000,2.0,3.333333,1.154701,3.20,1.000000,3.100000,1.354006,-1.20,0.233333,-0.354839,1.0,-1.333333,3.511885,1.00,7.293833,1.000000,7.393691,0.00,-2.333333,0.000000,0,0,1.572667,4,0.159250,1,3,0,False,10,0.0,0.0,0,0,21.42,43.140907,-21.720907
5858,2025-11-23,BOS,1,ORL,Neemias Queta,C,1,9.0,267.0,17,0,0,0,29.18,28.500000,11.484495,26.840,12.417143,25.580000,10.921615,2.340,2.920000,0.140735,0.0,0.000000,0.577350,0.4,1.224745,0.500000,1.414214,-0.4,-0.500000,-0.999998,5.0,4.000000,2.645751,3.20,2.408319,3.300000,1.776388,1.80,0.700000,0.515151,-5.0,5.333333,6.806859,8.80,7.694154,8.200000,8.796464,-13.80,-2.866667,-1.609756,0,0,2.920000,3,0.121583,1,0,0,False,2,0.0,0.0,0,0,5.85,27.224911,-21.374911
6082,2025-11-24,MEM,1,DEN,Zach Edey,C,0,-10.0,240.0,18,0,0,0,26.47,25.460000,5.600806,25.275,5.507928,25.275000,8.435954,1.195,0.185000,0.047280,4.0,2.333333,2.081666,2.0,1.788854,2.000000,1.567021,2.0,0.333333,1.000000,5.0,3.000000,2.081666,3.25,2.345208,3.250000,1.751190,1.75,-0.250000,0.538461,13.0,13.333333,5.131601,10.25,7.529940,10.250000,7.641989,2.75,3.083333,0.268293,0,0,0.185000,3,0.110292,1,0,0,False,2,0.0,0.0,0,0,5.68,26.8857

In [61]:
display(analyze_df_mins[(analyze_df_mins.Date == '2026-01-03') & (analyze_df_mins.Player == 'Alperen Sengun')])

,Date,Team,Team_type,Opp,Player,Pos,B2B,Spread,Total,team_game_num,cup_gm,pstszn_gm,is_OT,MP_lst_gm,MP_last_3_avg,MP_last_3_std,MP_last_5_avg,MP_last_5_std,MP_last_10_avg,MP_last_10_std,MP_change_L1,MP_change_L3,MP_pct_change,TOV_lst_gm,TOV_last_3_avg,TOV_last_3_std,TOV_last_5_avg,TOV_last_5_std,TOV_last_10_avg,TOV_last_10_std,TOV_change_L1,TOV_change_L3,TOV_pct_change,PF_lst_gm,PF_last_3_avg,PF_last_3_std,PF_last_5_avg,PF_last_5_std,PF_last_10_avg,PF_last_10_std,PF_change_L1,PF_change_L3,PF_pct_change,+/-_lst_gm,+/-_last_3_avg,+/-_last_3_std,+/-_last_5_avg,+/-_last_5_std,+/-_last_10_avg,+/-_last_10_std,+/-_change_L1,+/-_change_L3,+/-_pct_change,MP_spike,MP_drop,MP_trend,games_last_7_days,prev_team_mins_pct,role,missed_games,blowout,Injury_Ejection,DaysLstGm,travel_km,travel_hours,is_long_trip,same_arena,MP,MP_preds,Diff,Diff2
11156,2026-01-03,HOU,0,DAL,Alperen Sengun,C,0,-6.0,214.0,32,0,0,0,33.67,30.99,15.985607,32.848,12.391169,34.638,9.108929,0.822,-3.648,-0.027946,1.0,3.333333,0.57735,3.8,1.949359,3.6,1.619328,-2.8,-0.266667,-0.722222,4.0,2.333333,2.309401,2.8,2.04939,3.6,1.911951,1.2,-1.266667,0.111111,30.0,15.0,24.501701,14.4,21.714051,6.1,17.307994,15.6,8.9,3.918032,0,0,-3.648,1,0.140292,1,0,0,True,2,0.0,0.0,0,0,1.07,12.019753,-10.949753,10.949753


In [12]:
rmse = np.sqrt(mean_squared_error(y_test_mins, mins_preds)) # splits[5] = y_test
mae = mean_absolute_error(y_test_mins, mins_preds)
print('RMSE:', rmse)

df_yesterday = pd.read_csv(f'../tables/2025/gmday_preds_{tgt_stat}.csv')
df_yesterday['Date'] = pd.to_datetime(df_yesterday.Date)
df_yesterday = df_yesterday[(df_yesterday.Date == (datetime.strptime(now, "%Y-%m-%d") - timedelta(days=1)).strftime("%Y-%m-%d"))]\
                .rename(columns={"MP": "MP_proj"})

df_gms = pd.read_csv(f"../tables/2025/season_gamelogs.csv")
df_gms['Date'] = pd.to_datetime(df_gms.Date)

df_yesterday = df_yesterday.merge(df_gms[['Date', 'Team', 'Player', 'MP']], on=['Date', 'Team', 'Player'])
df_yesterday = df_yesterday[['Date', 'Team', 'Player', 'Pos', 'Opp', 'MP_proj', 'MP', 'MP_last_5_avg']][df_yesterday.MP > 0]

df_yesterday['Diff'] = abs(df_yesterday['MP_proj'] - df_yesterday['MP'])
df_yesterday['InRMSE_Range'] = np.where(df_yesterday['Diff'] <= rmse, 1, 0)

print("\nYesterday's Results:")
print("Total Accuracy (InRMSE_Range):", ((df_yesterday.InRMSE_Range == 1).sum() / df_yesterday.shape[0]))
print((df_yesterday.InRMSE_Range == 1).sum(), '/', df_yesterday.shape[0])

df_yesterday = df_yesterday.drop(['Diff'], axis=1)

if df_yesterday.shape[0] >= 50:
    for tm in df_yesterday.Team.unique():
        display(df_yesterday[df_yesterday.Team == tm])
else:
    display(df_yesterday)

RMSE: 5.692131078858187

Yesterday's Results:
Total Accuracy (InRMSE_Range): 0.8
64 / 80


,Date,Team,Player,Pos,Opp,MP_proj,MP,MP_last_5_avg,InRMSE_Range
0,2026-01-03,NYK,Karl-Anthony Towns,C,PHI,28.411846,33.62,31.899277,1
18,2026-01-03,NYK,Mikal Bridges,SF,PHI,35.643612,36.83,36.355669,1
27,2026-01-03,NYK,Tyler Kolek,PG,PHI,19.824503,13.57,21.131792,0
40,2026-01-03,NYK,OG Anunoby,PF,PHI,35.678345,37.25,34.496019,1
55,2026-01-03,NYK,Miles McBride,SG,PHI,29.077999,30.97,27.728961,1
69,2026-01-03,NYK,Jalen Brunson,PG,PHI,36.006405,34.63,36.650040,1


,Date,Team,Player,Pos,Opp,MP_proj,MP,MP_last_5_avg,InRMSE_Range
1,2026-01-03,SAS,Stephon Castle,PG,POR,32.328163,36.78,30.632796,1
11,2026-01-03,SAS,De'Aaron Fox,PG,POR,32.536491,34.52,30.925064,1
15,2026-01-03,SAS,Dylan Harper,SG,POR,22.118652,22.95,20.973669,1
63,2026-01-03,SAS,Julian Champagnie,SF,POR,27.097775,33.42,26.210842,0
64,2026-01-03,SAS,Keldon Johnson,SF,POR,23.595665,23.82,20.170511,1


,Date,Team,Player,Pos,Opp,MP_proj,MP,MP_last_5_avg,InRMSE_Range
2,2026-01-03,DAL,Max Christie,SG,HOU,29.052824,34.90,26.891979,0
7,2026-01-03,DAL,Anthony Davis,PF,HOU,34.038921,39.03,31.162738,1
26,2026-01-03,DAL,Cooper Flagg,PG,HOU,37.091766,34.62,35.560542,1
41,2026-01-03,DAL,Naji Marshall,SF,HOU,28.761560,33.28,31.297777,1
43,2026-01-03,DAL,P.J. Washington,PF,HOU,32.275539,12.00,31.248331,0


,Date,Team,Player,Pos,Opp,MP_proj,MP,MP_last_5_avg,InRMSE_Range
3,2026-01-03,MIA,Andrew Wiggins,SF,MIN,32.036499,28.02,29.900376,1
4,2026-01-03,MIA,Jaime Jaquez Jr.,SF,MIN,31.379396,9.95,29.122285,0
12,2026-01-03,MIA,Nikola Jovic,PF,MIN,23.177940,27.92,14.184584,1
65,2026-01-03,MIA,Norman Powell,SG,MIN,33.264751,24.58,30.163918,0
67,2026-01-03,MIA,Davion Mitchell,PG,MIN,29.313383,33.78,27.450117,1
79,2026-01-03,MIA,Bam Adebayo,C,MIN,32.347660,34.22,30.256756,1


...

,Date,Team,Player,Pos,Opp,MP_proj,MP,MP_last_5_avg,InRMSE_Range
22,2026-01-03,PHI,VJ Edgecombe,SG,NYK,36.369270,41.08,34.272108,1
37,2026-01-03,PHI,Joel Embiid,C,NYK,33.753849,33.08,29.897334,1
52,2026-01-03,PHI,Quentin Grimes,SG,NYK,31.665737,33.82,31.619585,1
62,2026-01-03,PHI,Paul George,PF,NYK,33.698509,35.17,31.542704,1
72,2026-01-03,PHI,Tyrese Maxey,PG,NYK,38.634117,37.42,37.939392,1


,Date,Team,Player,Pos,Opp,MP_proj,MP,MP_last_5_avg,InRMSE_Range
24,2026-01-03,POR,Shaedon Sharpe,SG,SAS,33.028393,28.32,31.405017,1
38,2026-01-03,POR,Deni Avdija,SF,SAS,36.292770,38.83,35.302800,1
48,2026-01-03,POR,Toumani Camara,PF,SAS,33.614189,35.78,32.182790,1


,Date,Team,Player,Pos,Opp,MP_proj,MP,MP_last_5_avg,InRMSE_Range
25,2026-01-03,CHO,Brandon Miller,SF,CHI,32.673813,36.33,33.007659,1
47,2026-01-03,CHO,Miles Bridges,PF,CHI,33.991241,35.33,32.410394,1
54,2026-01-03,CHO,Kon Knueppel,SF,CHI,32.169971,32.87,32.868275,1
76,2026-01-03,CHO,LaMelo Ball,PG,CHI,29.198853,26.43,28.219849,1


,Date,Team,Player,Pos,Opp,MP_proj,MP,MP_last_5_avg,InRMSE_Range
29,2026-01-03,BOS,Neemias Queta,C,LAC,26.719072,22.57,26.565491,1
36,2026-01-03,BOS,Jaylen Brown,SF,LAC,34.208282,35.13,32.109865,1
44,2026-01-03,BOS,Derrick White,SG,LAC,36.739292,37.03,34.553576,1
58,2026-01-03,BOS,Anfernee Simons,SG,LAC,26.084888,18.97,26.420839,0
73,2026-01-03,BOS,Payton Pritchard,PG,LAC,34.122993,39.63,33.021851,1


,Date,Team,Player,Pos,Opp,MP_proj,MP,MP_last_5_avg,InRMSE_Range
49,2026-01-03,HOU,Alperen Sengun,C,DAL,34.646507,1.07,31.741766,0
50,2026-01-03,HOU,Amen Thompson,SF,DAL,36.934551,39.37,34.282750,1
51,2026-01-03,HOU,Reed Sheppard,PG,DAL,28.971821,22.40,26.455751,0
68,2026-01-03,HOU,Kevin Durant,SF,DAL,36.931919,37.75,33.506115,1
71,2026-01-03,HOU,Jabari Smith Jr.,PF,DAL,35.127918,36.97,32.792640,1


# Main Model

In [28]:
def setup_df_main(df):
    
    # Stat dependent features 
    if tgt_stat == 'PTS':
        tgt_stat_cols = []
        df = df[['Season', 'Date', 'Team', 'Opp', 'Player', 'Pos', 'MP', 
         'PTS', 'TPM', 'FG', 'FGA', 'TPA', 'FT', 'FTA', 
         f'Off_{tgt_stat}', f'Off_L3_{tgt_stat}', f'Off_L5_{tgt_stat}', f'Off_L10_{tgt_stat}', f'Off_{tgt_stat}_Rk',
         f'Def_{tgt_stat}', f'Def_L3_{tgt_stat}', f'Def_L5_{tgt_stat}', f'Def_L10_{tgt_stat}', f'Def_{tgt_stat}_Rk',
         'Spread', 'Total', 'is_OT']]
        
        # Efficiency metrics
        df['three_rate_raw'] =  np.where(df.FGA > 0, df['TPA'] / df['FGA'], 0)
        df['ft_rate_raw']    =  np.where(df.FGA > 0, df['FTA'] / df['FGA'], 0)
        df['eFG_raw'] = (df['FG'] + 0.5 * df['TPM']) / df['FGA']
        df['TS_raw'] = df['PTS'] / (2 * (df['FGA'] + 0.44 * df['FTA']))    
        df['usage_proxy_raw'] =  np.where(df.MP > 0, (df['FGA'] + 0.44 * df['FTA']) / df['MP'], 0)
        
        for w in [3, 5, 10]:
            for metric in ['three_rate', 'ft_rate', 'eFG', 'TS', 'usage_proxy']:
                col = f"{metric}_L{w}"
                df[col] = (
                    df.groupby(['Player','Season'])[f'{metric}_raw']
                      .rolling(w, min_periods=1)
                      .mean()
                      .shift(1)
                      .reset_index(level=[0,1], drop=True)
                )
        for metric in ['three_rate', 'ft_rate', 'eFG', 'TS', 'usage_proxy']:
            col = f'{metric}_weighted'
            df[col] = (
                0.6 * df[f'{metric}_L3'] +
                0.3 * df[f'{metric}_L5'] +
                0.1 * df[f'{metric}_L10']
            )
            df = df.drop(f'{metric}_raw', axis=1)
        
    elif tgt_stat == 'PRA':
        tgt_stat_cols = ['PTS', 'REB', 'AST', 'TOV']
        df = df[['Season', 'Date', 'Team', 'Opp', 'Player', 'Pos', 'MP', 
         'PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK', 'TOV',
         'FG', 'FGA', 'TPA', 'FT', 'FTA', 
         f'Off_{tgt_stat}', f'Off_L3_{tgt_stat}', f'Off_L5_{tgt_stat}', f'Off_L10_{tgt_stat}', f'Off_{tgt_stat}_Rk',
         f'Def_{tgt_stat}', f'Def_L3_{tgt_stat}', f'Def_L5_{tgt_stat}', f'Def_L10_{tgt_stat}', f'Def_{tgt_stat}_Rk',
         'Spread', 'Total', 'is_OT']]
        
        df['usage_proxy_raw'] =  np.where(df.MP > 0, (df['FGA'] + 0.44 * df['FTA']) / df['MP'], 0)
        for w in [3, 5, 10]:
            df[f"usage_proxy_L{w}"] = (
                df.groupby(['Player','Season'])[f'usage_proxy_raw']
                  .rolling(w, min_periods=1)
                  .mean()
                  .shift(1)
                  .reset_index(level=[0,1], drop=True)
            )
        df['usage_proxy_weighted'] = (
            0.6 * df[f'usage_proxy_L3'] +
            0.3 * df[f'usage_proxy_L5'] +
            0.1 * df[f'usage_proxy_L10']
        )
        df = df.drop('usage_proxy_raw', axis=1)
        
        
    else:
        tgt_stat_cols = []
        df = df[['Season', 'Date', 'Team', 'Opp', 'Player', 'Pos', 'MP', 
         'PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK',
         'FG', 'FGA', 'TPA', 'FT', 'FTA', 
          f'Off_{tgt_stat}', f'Off_L3_{tgt_stat}', f'Off_L5_{tgt_stat}', f'Off_L10_{tgt_stat}', f'Off_{tgt_stat}_Rk',
          f'Def_{tgt_stat}', f'Def_L3_{tgt_stat}', f'Def_L5_{tgt_stat}', f'Def_L10_{tgt_stat}', f'Def_{tgt_stat}_Rk',
         'Spread', 'Total', 'is_OT']]

    
    # Create rolling + lag features    
    for col in ['MP', 'FGA', 'TPA', 'FTA'] + tgt_stat_cols:
        df[f'{col}_lst_gm'] = (
            df
            .groupby(['Player', 'Season'])[col]
            .shift(1)
        )
        for N in [3, 5, 10]:
            df[f'{col}_last_{N}_avg'] = (
                df.groupby(['Player', 'Season'])[col]
                  .rolling(window=N, min_periods=1)
                  .mean()
                  .shift(1)
                  .reset_index(level=[0, 1], drop=True)
            )
            df[f"{col}_last_{N}_std"] = (
                df.groupby(['Player', 'Season'])[col]
                  .shift(1)
                  .rolling(window=N, min_periods=1)
                  .std()
            )

    # Role identifiers features
    df['reserve_td'] = (df.MP < 8).astype(int)
    df['bench_td']   = ((df.MP >= 8) & (df.MP <= 25)).astype(int)
    df['starter_td'] = (df.MP > 25).astype(int)
    role_counts = df.groupby(['Season', 'Player'])[['reserve_td', 'bench_td', 'starter_td']].sum()
    role_counts['most_common_role'] = role_counts[['reserve_td', 'bench_td', 'starter_td']].idxmax(axis=1)
    role_counts['reserve'] = (role_counts['most_common_role'] == 'reserve_td').astype(int)
    role_counts['bench']   = (role_counts['most_common_role'] == 'bench_td').astype(int)
    role_counts['starter'] = (role_counts['most_common_role'] == 'starter_td').astype(int)
    df = df.merge(role_counts[['reserve', 'bench', 'starter']], on=['Season', 'Player'], how='left')
    df['role'] = 0
    df['role'] = np.where(df.starter == 1, 1, df.role)
    df['role'] = np.where(df.bench == 1, 2, df.role)
    df['role'] = np.where(df.reserve == 1, 3, df.role)
    
    # Other features
    df["DaysLstGm"] = (df.groupby("Player")["Date"].diff().dt.days).fillna(0).astype(int)
    
    for col in categories + ['Season', 'FG', 'FGA', 'FT', 'FTA', 'TPM', 'TPA', 
                             'reserve_td', 'reserve', 'bench_td', 'bench', 'starter_td', 'starter'] + tgt_stat_cols:
        if col == tgt_stat:
            continue
        if col in df.columns:
            df = df.drop(col, axis=1)
        
    return df

In [37]:
df_main = df[df.Season >= 2022].copy()
df_main = setup_df_main(df_main)
display(df_main)

n = len(df_main)
train_end = int(0.65 * n)
val_end   = int(0.85 * n)
main_train_df = df_main.iloc[:train_end]
main_val_df   = df_main.iloc[train_end:val_end]
main_test_df  = df_main.iloc[val_end:]
main_DFS = (main_train_df, main_val_df, main_test_df)

# stat_model = create_baseline_model(df_main, tgt_stat, main_DFS)
stat_model, stat_preds, y_test_stat, analyze_df_stat = hyperparam_tuning(main_DFS, tgt_stat, n_iter=1)
# feature_importance(stat_model)

# stat_model.save_model(f"../ML_models/{tgt_stat}_model.json")
# print(f'Saved {tgt_stat} model!')

,Date,Team,Opp,Player,Pos,MP,PRA,Off_PRA,Off_L3_PRA,Off_L5_PRA,Off_L10_PRA,Off_PRA_Rk,Def_PRA,Def_L3_PRA,Def_L5_PRA,Def_L10_PRA,Def_PRA_Rk,Spread,Total,is_OT,usage_proxy_L3,usage_proxy_L5,usage_proxy_L10,usage_proxy_weighted,MP_lst_gm,MP_last_3_avg,MP_last_3_std,MP_last_5_avg,MP_last_5_std,MP_last_10_avg,MP_last_10_std,FGA_lst_gm,FGA_last_3_avg,FGA_last_3_std,FGA_last_5_avg,FGA_last_5_std,FGA_last_10_avg,FGA_last_10_std,TPA_lst_gm,TPA_last_3_avg,TPA_last_3_std,TPA_last_5_avg,TPA_last_5_std,TPA_last_10_avg,TPA_last_10_std,FTA_lst_gm,FTA_last_3_avg,FTA_last_3_std,FTA_last_5_avg,FTA_last_5_std,FTA_last_10_avg,FTA_last_10_std,PTS_lst_gm,PTS_last_3_avg,PTS_last_3_std,PTS_last_5_avg,PTS_last_5_std,PTS_last_10_avg,PTS_last_10_std,REB_lst_gm,REB_last_3_avg,REB_last_3_std,REB_last_5_avg,REB_last_5_std,REB_last_10_avg,REB_last_10_std,AST_lst_gm,AST_last_3_avg,AST_last_3_std,AST_last_5_avg,AST_last_5_std,AST_last_10_avg,AST_last_10_std,TOV_lst_gm,TOV_last_3_avg,TOV_last_3_std,TOV_last_5_avg,TOV_last_5_std,TOV_last_10_avg,TOV_last_10_std,role,DaysLstGm
0,2022-10-18,1,22,19,0,23.10,12.0,12.000000,12.000000,12.0,12.0,4.0,16.000000,16.000000,16.0,16.0,2.0,9.0,243.0,0,0.340850,0.427717,0.394773,0.372302,NaN,23.850000,NaN,22.776,NaN,23.716,NaN,NaN,6.333333,NaN,8.2,NaN,8.0,NaN,NaN,1.666667,NaN,2.4,NaN,2.4,NaN,NaN,4.000000,NaN,3.2,NaN,2.9,NaN,NaN,12.666667,NaN,13.4,NaN,12.2,NaN,NaN,3.000000,NaN,3.8,NaN,3.8,NaN,NaN,4.333333,NaN,3.6,NaN,3.7,NaN,NaN,1.666667,NaN,1.6,NaN,1.2,NaN,1,0
1,2022-10-18,1,22,71,0,8.28,7.0,7.000000,7.000000,7.0,7.0,5.0,16.000000,16.000000,16.0,16.0,2.0,9.0,243.0,0,0.302402,0.286831,0.167386,0.284229,NaN,5.410000,NaN,3.964,NaN,6.374,NaN,NaN,1.333333,NaN,0.8,NaN,0.6,NaN,NaN,0.000000,NaN,0.0,NaN,0.0,NaN,NaN,0.000000,NaN,0.4,NaN,0.2,NaN,NaN,2.000000,NaN,1.4,NaN,0.9,NaN,NaN,0.333333,NaN,0.4,NaN,0.7,NaN,NaN,0.333333,NaN,0.2,NaN,0.3,NaN,NaN,0.333333,NaN,0.2,NaN,0.3,NaN,2,0
2,2022-10-18,1,22,231,4,24.03,9.0,9.000000,9.000000,9.0,9.0,5.0,9.000000,9.000000,9.0,9.0,1.0,9.0,243.0,0,0.321647,0.371026,0.392727,0.343569,NaN,11.733333,NaN,13.474,NaN,12.434,NaN,NaN,3.666667,NaN,4.8,NaN,4.7,NaN,NaN,0.333333,NaN,0.8,NaN,1.1,NaN,NaN,0.000000,NaN,0.6,NaN,0.6,NaN,NaN,4.000000,NaN,6.0,NaN,5.8,NaN,NaN,1.000000,NaN,2.2,NaN,1.6,NaN,NaN,2.333333,NaN,3.0,NaN,2.8,NaN,NaN,2.000000,NaN,1.2,NaN,1.0,NaN,1,0
3,2022-10-18,1,22,317,1,23.95,17.0,17.000000,17.000000,17.0,17.0,2.0,17.000000,17.000000,17.0,17.0,2.0,9.0,243.0,0,0.394162,0.359923,0.378902,0.382364,NaN,14.130000,NaN,15.152,NaN,15.162,NaN,NaN,4.666667,NaN,4.8,NaN,5.2,NaN,NaN,2.666667,NaN,2.2,NaN,2.8,NaN,NaN,2.666667,NaN,1.8,NaN,1.3,NaN,NaN,7.333333,NaN,6.2,NaN,5.7,NaN,NaN,2.333333,NaN,2.6,NaN,1.9,NaN,NaN,0.000000,NaN,0.8,NaN,0.7,NaN,NaN,0.666667,NaN,0.4,NaN,0.5,NaN,1,0
4,2022-10-18,1,22,431,3,38.57,40.0,40.000000,40.000000,40.0,40.0,2.0,91.000000,91.000000,91.0,91.0,3.0,9.0,243.0,0,0.322357,0.322357,0.322357,0.322357,NaN,15.055000,NaN,15.055,NaN,15.055,NaN,NaN,3.500000,NaN,3.5,NaN,3.5,NaN,NaN,1.000000,NaN,1.0,NaN,1.0,NaN,NaN,1.000000,NaN,1.0,NaN,1.0,NaN,NaN,6.500000,NaN,6.5,NaN,6.5,NaN,NaN,3.000000,NaN,3.0,NaN,3.0,NaN,NaN,0.500000,NaN,0.5,NaN,0.5,NaN,NaN,1.500000,NaN,1.5,NaN,1.5,NaN,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84482,2026-01-03,28,9,581,0,18.67,9.0,17.906250,13.000000,17.4,16.7,33.0,32.741935,19.333333,16.6,25.0,16.0,-9.0,237.0,0,0.502506,0.482506,0.392727,0.485528,28.42,21.460000,11.197055,21.984,9.038716,22.710,9.867945,13.0,9.333333,8.621678,9.4,6.730527,8.1,6.142746,5.0,4.000000,4.041452,3.4,3.464102,2.7,2.869379,0.0,2.000000,3.214550,2.8,2.387467,2.4,2.796824,4.0,6.666667,16.921387,9.4,12.008330,9.1,10.870960,6.0,4.000000,0.577350,5.8,2.607681,6.1,3.705851,1.0,2.333333,3.055050,2.2,4.207137,1.5,


Trial 1/1: {'n_estimators': 914, 'learning_rate': np.float64(0.020362600399019493), 'max_depth': 3, 'min_child_weight': 4, 'subsample': np.float64(0.8777772268030892), 'colsample_bytree': np.float64(0.9596576607390586), 'gamma': np.float64(0.4648492008890568), 'reg_lambda': np.float64(1.7176692635574076), 'reg_alpha': np.float64(0.057434043104525045)}
Validation MAE: 2.9830

Best validation MAE: 2.982966390581408
Best parameters: {'learning_rate': np.float64(0.020362600399019493), 'max_depth': 3, 'min_child_weight': 4, 'subsample': np.float64(0.8777772268030892), 'colsample_bytree': np.float64(0.9596576607390586), 'gamma': np.float64(0.4648492008890568), 'reg_lambda': np.float64(1.7176692635574076), 'reg_alpha': np.float64(0.057434043104525045), 'objective': 'reg:squarederror', 'tree_method': 'hist', 'device': 'cuda', 'seed': 42}

Test Metrics:
RMSE: 3.9466764740608187
MAE: 2.982517451767442
R²: 0.9003708238079291


In [15]:
# analyze_df_stat = main_test_df.drop([tgt_stat, f'{tgt_stat}_preds'], axis=1)\
#                 .merge(analyze_df_stat[['Date', 'Team', 'Player', tgt_stat, f'{tgt_stat}_preds']], on=['Date', 'Team', 'Player'])
# analyze_df_stat['Diff'] = analyze_df_stat[tgt_stat] - analyze_df_stat[f'{tgt_stat}_preds']
# analyze_df_stat['Diff2'] = abs(analyze_df_stat['Diff'])
# analyze_df_stat[analyze_df_stat.MP > 38].sort_values('Diff2', ascending=False).drop('Diff2', axis=1).head(15)

In [33]:
rmse = np.sqrt(mean_squared_error(y_test_stat, stat_preds)) # splits[5] = y_test
mae = mean_absolute_error(y_test_stat, stat_preds)
print('RMSE:', rmse)

df_yesterday = pd.read_csv(f'../tables/2025/gmday_preds_{tgt_stat}.csv')
df_yesterday['Date'] = pd.to_datetime(df_yesterday.Date)
df_yesterday = df_yesterday[(df_yesterday.Date == (datetime.strptime(now, "%Y-%m-%d") - timedelta(days=1)).strftime("%Y-%m-%d"))]\
                .rename(columns={"MP": "MP_proj"})

df_gms = pd.read_csv(f"../tables/2025/season_gamelogs.csv")
df_gms['Date'] = pd.to_datetime(df_gms.Date)
df_gms = df_gms.rename(columns={"TRB": "REB", "3PM": "TPM", "3PA": "TPA"})
df_gms['STL_BLK'] = df_gms.STL + df_gms.BLK
df_gms['PR'] = df_gms.PTS + df_gms.REB 
df_gms['PA'] = df_gms.PTS + df_gms.AST
df_gms['RA'] = df_gms.REB + df_gms.AST
df_gms['PRA'] = df_gms.PTS + df_gms.REB + df_gms.AST

df_yesterday = df_yesterday.merge(df_gms[['Date', 'Team', 'Player', tgt_stat, 'MP']], on=['Date', 'Team', 'Player'])
df_yesterday = df_yesterday[['Date', 'Team', 'Player', 'Pos', 'Opp', 'MP_proj', 'MP', f'{tgt_stat}_line', f'{tgt_stat}_proj', tgt_stat]][df_yesterday.MP > 0]

df_yesterday['Diff'] = df_yesterday[f'{tgt_stat}_proj'] - df_yesterday[f'{tgt_stat}_line']
df_yesterday['Diff2'] = abs(df_yesterday[f'{tgt_stat}_proj'] - df_yesterday[tgt_stat])
df_yesterday['Act_Res'] = np.where(df_yesterday[tgt_stat] > df_yesterday[f'{tgt_stat}_line'], 'O', 'U')
df_yesterday['Pred_Res'] = np.where(df_yesterday[f'{tgt_stat}_proj'] > df_yesterday[f'{tgt_stat}_line'], 'O', 'U')
df_yesterday['ParlayHit'] = np.where(df_yesterday['Act_Res'] == df_yesterday['Pred_Res'], 1, 0)
df_yesterday['InRMSE_Range'] = np.where(df_yesterday['Diff2'] <= rmse, 1, 0)

print("Total Accuracy (ParlayHit):", ((df_yesterday.ParlayHit == 1).sum() / df_yesterday.shape[0]))
print((df_yesterday.ParlayHit == 1).sum(), "/", df_yesterday.shape[0])

print("\nTotal Accuracy (InRMSE_Range):", ((df_yesterday.InRMSE_Range == 1).sum() / df_yesterday.shape[0]))
print((df_yesterday.InRMSE_Range == 1).sum(), "/", df_yesterday.shape[0])

df_yesterday = df_yesterday.drop(['Diff', 'Act_Res', 'Pred_Res'], axis=1).sort_values(f'{tgt_stat}_line', ascending=False)

if df_yesterday.shape[0] >= 50:
    for tm in df_yesterday.Team.unique():
        display(df_yesterday[(df_yesterday.Team == tm)]) #  & (df_yesterday.ParlayHit == 1)
else:
    display(df_yesterday)

RMSE: 3.9318102643950072
Total Accuracy (ParlayHit): 0.55
44 / 80

Total Accuracy (InRMSE_Range): 0.3125
25 / 80


,Date,Team,Player,Pos,Opp,MP_proj,MP,PRA_line,PRA_proj,PRA,Diff2,ParlayHit,InRMSE_Range
42,2026-01-03,ATL,Jalen Johnson,SF,TOR,36.261528,37.15,44.5,42.257584,46,3.742416,0,1
8,2026-01-03,ATL,Onyeka Okongwu,C,TOR,34.439484,30.30,29.5,35.062859,11,24.062859,0,0
70,2026-01-03,ATL,Nickeil Alexander-Walker,SG,TOR,33.597206,35.93,26.5,27.186911,38,10.813089,1,0
33,2026-01-03,ATL,Dyson Daniels,SG,TOR,35.571247,34.23,24.5,27.408985,37,9.591015,1,0
32,2026-01-03,ATL,Zaccharie Risacher,SF,TOR,23.109283,30.17,15.5,12.496202,15,2.503798,1,1


,Date,Team,Player,Pos,Opp,MP_proj,MP,PRA_line,PRA_proj,PRA,Diff2,ParlayHit,InRMSE_Range
38,2026-01-03,POR,Deni Avdija,SF,SAS,36.292770,38.83,42.5,40.068214,50,9.931786,0,0
24,2026-01-03,POR,Shaedon Sharpe,SG,SAS,33.028393,28.32,31.5,27.938684,12,15.938684,1,0
48,2026-01-03,POR,Toumani Camara,PF,SAS,33.614189,35.78,21.5,19.806179,29,9.193821,0,0


,Date,Team,Player,Pos,Opp,MP_proj,MP,PRA_line,PRA_proj,PRA,Diff2,ParlayHit,InRMSE_Range
69,2026-01-03,NYK,Jalen Brunson,PG,PHI,36.006405,34.63,40.5,39.712711,39,0.712711,1,1
0,2026-01-03,NYK,Karl-Anthony Towns,C,PHI,28.411846,33.62,36.5,55.712944,39,16.712944,1,0
40,2026-01-03,NYK,OG Anunoby,PF,PHI,35.678345,37.25,27.5,25.116663,31,5.883337,0,0
18,2026-01-03,NYK,Mikal Bridges,SF,PHI,35.643612,36.83,26.5,22.472406,23,0.527594,1,1
55,2026-01-03,NYK,Miles McBride,SG,PHI,29.077999,30.97,18.5,19.833128,27,7.166872,1,0
27,2026-01-03,NYK,Tyler Kolek,PG,PHI,19.824503,13.57,11.5,14.783369,9,5.783369,0,0


,Date,Team,Player,Pos,Opp,MP_proj,MP,PRA_line,PRA_proj,PRA,Diff2,ParlayHit,InRMSE_Range
36,2026-01-03,BOS,Jaylen Brown,SF,LAC,34.208282,35.13,40.5,43.110752,58,14.889248,1,0
44,2026-01-03,BOS,Derrick White,SG,LAC,36.739292,37.03,28.5,30.686228,40,9.313772,1,0
73,2026-01-03,BOS,Payton Pritchard,PG,LAC,34.122993,39.63,25.5,25.971193,24,1.971193,0,1
29,2026-01-03,BOS,Neemias Queta,C,LAC,26.719072,22.57,19.5,16.277025,18,1.722975,1,1
58,2026-01-03,BOS,Anfernee Simons,SG,LAC,26.084888,18.97,16.5,17.812746,22,4.187254,1,0


...

,Date,Team,Player,Pos,Opp,MP_proj,MP,PRA_line,PRA_proj,PRA,Diff2,ParlayHit,InRMSE_Range
7,2026-01-03,DAL,Anthony Davis,PF,HOU,34.038921,39.03,34.5,40.307499,41,0.692501,1,1
26,2026-01-03,DAL,Cooper Flagg,PG,HOU,37.091766,34.62,29.5,32.852634,23,9.852634,0,0
43,2026-01-03,DAL,P.J. Washington,PF,HOU,32.275539,12.00,21.5,19.304811,1,18.304811,1,0
41,2026-01-03,DAL,Naji Marshall,SF,HOU,28.761560,33.28,15.5,17.793915,17,0.793915,1,1
2,2026-01-03,DAL,Max Christie,SG,HOU,29.052824,34.90,13.5,20.582405,33,12.417595,1,0


,Date,Team,Player,Pos,Opp,MP_proj,MP,PRA_line,PRA_proj,PRA,Diff2,ParlayHit,InRMSE_Range
76,2026-01-03,CHO,LaMelo Ball,PG,CHI,29.198853,26.43,33.5,33.309704,30,3.309704,1,1
25,2026-01-03,CHO,Brandon Miller,SF,CHI,32.673813,36.33,29.5,32.882801,32,0.882801,1,1
54,2026-01-03,CHO,Kon Knueppel,SF,CHI,32.169971,32.87,28.5,27.127329,24,3.127329,1,1
47,2026-01-03,CHO,Miles Bridges,PF,CHI,33.991241,35.33,28.5,26.469828,41,14.530172,0,0


,Date,Team,Player,Pos,Opp,MP_proj,MP,PRA_line,PRA_proj,PRA,Diff2,ParlayHit,InRMSE_Range
1,2026-01-03,SAS,Stephon Castle,PG,POR,32.328163,36.78,33.5,24.670403,30,5.329597,1,0
11,2026-01-03,SAS,De'Aaron Fox,PG,POR,32.536491,34.52,32.5,27.370874,29,1.629126,1,1
64,2026-01-03,SAS,Keldon Johnson,SF,POR,23.595665,23.82,21.5,20.352251,14,6.352251,1,0
15,2026-01-03,SAS,Dylan Harper,SG,POR,22.118652,22.95,21.5,16.885052,12,4.885052,1,0
63,2026-01-03,SAS,Julian Champagnie,SF,POR,27.097775,33.42,20.5,19.337337,34,14.662663,0,0


,Date,Team,Player,Pos,Opp,MP_proj,MP,PRA_line,PRA_proj,PRA,Diff2,ParlayHit,InRMSE_Range
35,2026-01-03,CHI,Nikola Vucevic,C,CHO,32.098225,33.98,31.5,28.640471,43,14.359529,0,0
45,2026-01-03,CHI,Matas Buzelis,PF,CHO,27.605762,34.35,24.5,22.370392,28,5.629608,0,0
59,2026-01-03,CHI,Tre Jones,PG,CHO,27.599106,27.93,23.5,22.189377,20,2.189377,1,1
39,2026-01-03,CHI,Jalen Smith,C,CHO,23.103416,15.02,23.5,21.088161,15,6.088161,1,0
17,2026-01-03,CHI,Ayo Dosunmu,SG,CHO,24.674477,27.88,22.5,18.130354,24,5.869646,0,0
19,2026-01-03,CHI,Kevin Huerter,SF,CHO,23.162807,30.53,20.5,16.613577,27,10.386423,0,0
61,2026-01-03,CHI,Isaac Okoro,SG,CHO,24.711365,27.17,14.5,15.674969,5,10.674969,0,0


,Date,Team,Player,Pos,Opp,MP_proj,MP,PRA_line,PRA_proj,PRA,Diff2,ParlayHit,InRMSE_Range
65,2026-01-03,MIA,Norman Powell,SG,MIN,33.264751,24.58,30.5,31.569702,23,8.569702,0,0
79,2026-01-03,MIA,Bam Adebayo,C,MIN,32.347660,34.22,28.5,28.493401,25,3.493401,1,1
4,2026-01-03,MIA,Jaime Jaquez Jr.,SF,MIN,31.379396,9.95,24.5,30.989481,10,20.989481,0,0
3,2026-01-03,MIA,Andrew Wiggins,SF,MIN,32.036499,28.02,22.5,29.318312,17,12.318312,0,0
67,2026-01-03,MIA,Davion Mitchell,PG,MIN,29.313383,33.78,17.5,16.498808,23,6.501192,0,0
12,2026-01-03,MIA,Nikola Jovic,PF,MIN,23.177940,27.92,17.5,22.472679,21,1.472679,1,1


### Today's predictions

In [64]:
df_lines = pd.read_csv(f"../tables/2025/parlay_lines.csv")
df_lines['Date'] = pd.to_datetime(df_lines.Date)
df_lines = df_lines[~(df_lines.Team.isnull())]

# Predict Mins
df_lines["Team"] = team_encoder.transform(df_lines["Team"])
df_pred = df_pred.merge(df_lines[['Date', 'Team', 'Spread', 'Total']], on=['Date', 'Team'], how='left')
df_pred = df_pred[~df_pred[['Date', 'Team', 'Player']].duplicated(keep='last')]
df_pred['Spread_x'] = np.where(df_pred.Spread_x.isnull(), df_pred.Spread_y, df_pred.Spread_x)
df_pred['Total_x'] = np.where(df_pred.Total_x.isnull(), df_pred.Total_y, df_pred.Total_x)
df_pred = df_pred.rename(columns={"Spread_x": "Spread", "Total_x": "Total"}).drop(['Spread_y', 'Total_y'], axis=1)
df_pred_mins = setup_df_mins(con, df_pred)
df_pred_mins = df_pred_mins.drop(['Date', 'MP'], axis=1)
DM_mins = xgb.DMatrix(df_pred_mins)
df_pred['MP'] = mins_model.predict(DM_mins)

# Predict Stat
df_pred = setup_df_main(df_pred)
feature_cols = [col for col in df_pred.columns if col not in ['Date', tgt_stat]]
df_pred = df_pred[df_pred.Date == now][feature_cols]
DM_stats = xgb.DMatrix(df_pred)
df_pred[f"{tgt_stat}_proj"] = stat_model.predict(DM_stats)

df_pred['Team'] = team_encoder.inverse_transform(df_pred["Team"])
df_lines['Team'] = team_encoder.inverse_transform(df_lines["Team"])
df_pred['Opp'] = team_encoder.inverse_transform(df_pred["Opp"])
df_pred['Player'] = player_encoder.inverse_transform(df_pred["Player"])
df_pred['Pos'] = position_encoder.inverse_transform(df_pred["Pos"])

df_lines = df_lines[df_lines.Date == now][['Team', 'Player', f'{tgt_stat}_line']]
df_pred = df_pred.merge(df_lines, on=['Team', 'Player'])

tds_picks = df_pred[~(df_pred[f'{tgt_stat}_line'].isnull())]\
            [['Team', 'Player', 'Pos', 'Opp', 'MP', 'MP_last_5_avg', f'{tgt_stat}_line', f'{tgt_stat}_proj']]
tds_picks['Diff'] = abs((df_pred[f'{tgt_stat}_line'] - df_pred[f'{tgt_stat}_proj']))
tds_picks['Diff2'] = abs((df_pred['MP'] - df_pred['MP_last_5_avg']))
tds_picks = tds_picks.sort_values('Diff', ascending=False).drop(['Diff', 'Diff2'], axis=1)
if tds_picks.shape[0] >= 50:
    print(tds_picks.shape[0], 'rows')
    for tm in tds_picks.Team.unique():
        display(tds_picks[tds_picks.Team == tm])
else:
    display(tds_picks)
tds_picks.insert(0, 'Date', pd.to_datetime(now))
# partition_save_df(tds_picks, f"../tables/2025/gmday_preds_{tgt_stat}.csv")

75 rows


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PRA_line,PRA_proj
59,MIL,Giannis Antetokounmpo,PF,SAC,28.571327,23.713046,45.5,33.176754
81,MIL,Myles Turner,C,SAC,27.891743,28.751699,17.5,21.150198
12,MIL,Ryan Rollins,PG,SAC,32.128468,30.049000,25.5,27.504063
9,MIL,Kevin Porter Jr.,PG,SAC,37.126045,36.197249,29.5,31.107489
67,MIL,Kyle Kuzma,PF,SAC,30.262310,29.384978,18.5,17.401583
66,MIL,Bobby Portis,PF,SAC,24.127415,26.353123,19.5,20.189936


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PRA_line,PRA_proj
17,DET,Marcus Sasser,PG,CLE,14.255999,7.612516,17.5,9.462994
89,DET,Isaiah Stewart,C,CLE,24.571085,22.763269,21.5,14.767603
50,DET,Ausar Thompson,SF,CLE,27.505112,26.914248,21.5,15.236931
52,DET,Duncan Robinson,SF,CLE,26.348499,26.803595,16.5,12.903678
41,DET,Jaden Ivey,SG,CLE,18.983414,17.494018,15.5,12.904775
2,DET,Cade Cunningham,PG,CLE,36.036301,35.032391,44.5,45.837540


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PRA_line,PRA_proj
24,SAC,Zach LaVine,SG,MIL,29.144629,30.277499,21.5,28.366283
14,SAC,Russell Westbrook,PG,MIL,28.156128,29.209290,27.5,22.922308
15,SAC,Dennis Schroder,PG,MIL,26.053459,25.653917,19.5,20.742168
64,SAC,DeMar DeRozan,PF,MIL,33.633686,31.055751,27.5,27.414207


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PRA_line,PRA_proj
32,CLE,Sam Merrill,SG,DET,22.668297,22.345003,14.5,21.263771
73,CLE,Evan Mobley,PF,DET,30.978827,30.440726,29.5,25.038359
88,CLE,Jarrett Allen,C,DET,27.451883,26.322117,22.5,26.661297
23,CLE,Donovan Mitchell,SG,DET,34.704842,33.533903,38.5,34.857674
46,CLE,De'Andre Hunter,SF,DET,26.756336,26.294686,19.5,17.354872
76,CLE,Dean Wade,PF,DET,17.116302,23.133262,11.5,9.756242
31,CLE,Jaylon Tyson,SG,DET,26.613388,27.600388,20.5,18.813086
11,CLE,Darius Garland,PG,DET,32.567997,32.194897,27.5,28.919880


...

,Team,Player,Pos,Opp,MP,MP_last_5_avg,PRA_line,PRA_proj
85,MEM,Jaren Jackson Jr.,C,LAL,33.099068,31.906249,28.5,32.520462
33,MEM,Cedric Coward,SG,LAL,26.890629,22.940564,23.5,27.284685
3,MEM,Ja Morant,PG,LAL,30.759323,28.179024,35.5,33.651196
34,MEM,Jaylen Wells,SG,LAL,30.745659,31.977370,18.5,19.602957


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PRA_line,PRA_proj
70,IND,Pascal Siakam,PF,ORL,34.454170,31.430051,35.5,31.748276
5,IND,Andrew Nembhard,PG,ORL,33.924873,31.967221,28.5,25.377476
10,IND,T.J. McConnell,PG,ORL,17.462122,16.864971,17.5,14.971234
48,IND,Aaron Nesmith,SF,ORL,27.989170,23.916338,19.5,19.924664


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PRA_line,PRA_proj
18,ORL,Tyus Jones,PG,IND,21.425209,23.956365,13.5,10.157410
27,ORL,Desmond Bane,SG,IND,35.663246,35.762905,31.5,29.098343
8,ORL,Anthony Black,PG,IND,35.344437,34.905561,28.5,30.683262
82,ORL,Wendell Carter Jr.,C,IND,32.317131,31.589875,23.5,21.630550
62,ORL,Paolo Banchero,PF,IND,36.316105,34.561938,39.5,38.115013


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PRA_line,PRA_proj
44,BRK,Michael Porter Jr.,SF,DEN,32.247471,32.766962,38.5,35.739426
39,BRK,Terance Mann,SG,DEN,26.210239,27.038805,13.5,13.474676


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PRA_line,PRA_proj
54,MIA,Jaime Jaquez Jr.,SF,NOP,28.905888,25.830143,26.5,29.146309
16,MIA,Davion Mitchell,PG,NOP,31.934984,28.061951,20.5,22.786522
53,MIA,Andrew Wiggins,SF,NOP,31.155277,30.251920,25.5,27.756023
19,MIA,Norman Powell,SG,NOP,32.358200,31.095859,31.5,30.391956
78,MIA,Bam Adebayo,C,NOP,32.837025,30.661516,30.5,29.938915
